# An Exploratory Analysis of the Titanic Dataset

This project builds a function to predict a Titanic passenger’s survival probability by following these steps:

1. **Data Preparation:** Clean the data and encode categorical features.
2. **Exploratory Analysis:** Examine relationships between features and survival.
3. **Model Training:** Train a classification model using the prepared data.
4. **Prediction Function:** Apply the trained model to new passenger inputs to estimate survival probability.

In [ ]:
# Import necessary modules
import pandas as pd                # For handling data
import matplotlib.pyplot as plt    # For visualisations